In [1]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas


In [38]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

awg.awg._ch[1].sync_marker_high=1.5
awg.awg._ch[1].sync_marker_low = 0

awg.awg._ch[2].sample_marker_low = 0
awg.awg._ch[2].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[2].output_amplitude = 0.5

awg.awg._ch[2].sync_marker_high=1.5
awg.awg._ch[2].sync_marker_low = 0


In [37]:
awg.awg._ch[1].sync_marker_offset

0.0

In [176]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5


seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})
seq.awg =awg
seq.start_new_segment('Starting segment')


seq.asc(length_mus = 10, green=True, orange = True)


seq.sequences['awg'][1].print_info()

0     Laser             10.000000
   0     Starting segment  10.000000
      0                       10.000000wait    1       0       
      1     _missing_smpls_   0.000000wait    0       0       


In [241]:
seq.write_seq()

MCAS 'Laser' has been generated.


In [255]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0160 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0130 seconds


In [256]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0160 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0100 seconds


## CPMG

In [39]:
[1,400e-3]
0.5,200e-3

(0.5, 0.2)

In [40]:
AWG_M8190A_qudi.awg._ch[1].output_amplitude

0.5

In [3]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5



def awg_AddReadout(seq):
    length_laserreadout_ns = 3000
    length_lasersafety_ns = 1000
    length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    
    while length_laserreadout_smpl % 64 != 0:
        length_laserreadout_smpl += 1
        
    seq.asc(length_mus=0.5)  # 2nd step
    seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)
    
    seq.asc(length_mus= length_lasersafety_ns / 1e3)
    return seq

def awg_AddWait(seq, time, orange=False, green=False):
    enable_reuse_segments = False
    if time < 0.:
        for i in range(3): 
            print ('Negative time for sequence in AWG!!!')
    elif time > 10.:
        if int(time / (32 / 12.)) >= 1:
            seq.start_new_segment('wait_loop', reuse_segment=enable_reuse_segments,
                                  loop_count=int(time / (32 / 12.)))
            seq.asc(length_smpl=32000, orange=orange, green=green)
        seq.start_new_segment('wait_additional')
        seq.asc(length_mus=time % (32 / 12.), orange=orange, green=green)
    else:
        seq.asc(length_mus=time, orange=orange, green=green)
    return seq

def AddRfSafety(seq):
    rf_safety = 1
    seq.asc(length_mus=rf_safety)
    return seq



def AddPulseCh1( seq, freq, amp, phase, square=None, length_us=None, length_smpl=None):
    electron_usegauss = False
    if length_us is not None:
        length_smpl = length_us / 0.0000833333333
    if square is None:
        square = not (electron_usegauss)
    if square:
        seq.asc(pdawg1=dict(type='sine', frequencies=[freq], amplitudes=[amp], phases=[phase]),
                length_smpl=length_smpl)
    else:
        # seq.AddGaussianPulse_samples(length_smpl,freq,amp,phase,sigma_to_duration_ratio=1/4.)
        seq.asc(pdawg1=dict(type='sinegauss', frequencies=[freq], amplitudes=[amp], phases=[phase]),
                length_smpl=length_smpl)
    return seq







def awg_hahn_echo(taus):
    seq = mcas.MultiChSeq('hahn_echo', ch_dict={'awg': [1, 2]})
    phases = [0.,180.]
    awg_freq = 2827.259  # in MHz
    electron_amp = 0.2 # AU 300mV peak to peak
    electron_pi = 1.4/2 # us
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
    for tau in taus:
        for phase in phases:

            seq.start_new_segment('hahn')
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                    length_mus=electron_pi / 2.)
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],
                                phases=[phase + (-360.) * oscReadoutPhaseFreq * tau]),
                    length_mus=electron_pi / 2.)
            seq.asc(length_mus=0.1)
            
#             seq = AddRfSafety(seq)
            seq = awg_AddReadout(seq)
    return seq




# def awg_cpmg_detection( taus):
#     angles = [90, 90]
#     phases = [0.,180.]
#     awg_freq = 2827.259  # in MHz
#     electron_amp = 0.2 # AU 300mV peak to peak
#     electron_pi = 1.4/2 # us
#     oscReadoutPhaseFreq = 0
#     nitrogen_transition = 14.0733
#     rf_safety = 1
#     tau_probe = 100.

#     seq = mcas.MultiChSeq('cpmg', ch_dict={'awg': [1, 2]})

#     for tau in taus:
#         pi_samples = round(electron_pi * 1e-6 / (1. / 12e9))
#         pi_half_samples = round(electron_pi / 2. * 1e-6 / (1. / 12e9))
#         tau_samples =  round(tau * 1e-6 / (1. / 12e9))
#         wait = tau_samples - pi_samples
#         wait_half = wait / 2.

#         if wait_half % 1 == 0:
#             wait_half_2 = wait_half
#         else:
#             wait_half_2 = int(wait_half) + 1
#             wait_half = int(wait_half)
        
#         # iterations=int(self.tau_probe/(20.*(tau/12e3)))
#         iterations = int(tau_probe)

#         wait_short = int(wait)
#         wait_sample = wait - wait_short

#         all_angles = []
#         all_waits = []
#         m = 0.

#         for iteration in range(iterations):
#             for angle in angles:
#                 all_angles.append(float(angle))
#                 m += wait_sample
#                 if np.abs(m) <= 0.5:
#                     all_waits.append(wait_short)
#                 else:
#                     all_waits.append(wait_short + 1)
#                     m -= 1.

#         for phase in phases:
#             seq.start_new_segment('cpmg')

#             seq = AddPulseCh1(seq, awg_freq, electron_amp, 0., length_smpl=pi_half_samples)
#             seq.asc(length_smpl=wait_half)
#             for i, angle in enumerate(all_angles):
#                 if i < len(all_angles) - 1:
#                     seq = AddPulseCh1(seq, awg_freq, electron_amp, angle,
#                                            length_smpl=pi_samples)
#                     seq.asc(length_smpl=all_waits[i])
#                 else:
#                     seq = AddPulseCh1(seq, awg_freq, electron_amp, angle,
#                                            length_smpl=pi_samples)
#             seq.asc(length_smpl=wait_half_2)
#             # seq.asc(pdawg1=dict(type='sine',frequencies=[self.awg_freq],amplitudes=[self.electron_amp_fast]),
#             # phases=[phase],length_smpl=pi_half_samples)
#             seq = AddPulseCh1(seq, awg_freq, electron_amp, phase, length_smpl=pi_half_samples)
#             seq.asc(length_mus=1)
#             seq = awg_AddReadout(seq)

# #             seq = self.AddNitrogenPi(seq)
# #             seq = self.AddRfSafety(seq)
# #             seq = self.awg_AddSsr(seq)

#     return seq





In [12]:

def awg_cpmg( taus):
    
    seq = mcas.MultiChSeq('my_cpmg', ch_dict={'awg': [1, 2]})
    phases = [0.,180.]
    
    
    awg_freq = 2329.92 # in MHz
    electron_amp = 0.2 # AU 300mV peak to peak
    electron_pi = 1.436/2    
    
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
#     double check
    
    for tau in taus:
        for phase in phases:

            seq.start_new_segment('my_cpmg')
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                    length_mus=electron_pi / 2.)
# 1X
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)
# 2Y          
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)
# 3X            
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)
# 4Y            
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)
# 5X            

            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)                
# 6Y 
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)            
# 7X 
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)                
# 8Y
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)            
#RO    
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],
                                phases=[phase]),
                    length_mus=electron_pi / 2.)
            seq.asc(length_mus=0.1)          
            seq = awg_AddReadout(seq)
    return seq


In [13]:
# taus = np.arange(45., 60, 0.05)
taus = np.arange(1., 30, 1.5)

seqCPMG = awg_cpmg(taus)
# seqCPMG = awg_cpmg_detection(taus)

seqCPMG.awg = awg

pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= taus*1e-6,
                                           number_of_lasers=2*len(taus), 
                                           laser_ignore_list=[], 
                                           alternating=True, 
                                           units=('s', 'arb. u.'))

In [14]:
len(taus)

20

In [15]:
taus

array([ 1. ,  2.5,  4. ,  5.5,  7. ,  8.5, 10. , 11.5, 13. , 14.5, 16. , 17.5, 19. , 20.5, 22. , 23.5, 25. , 26.5, 28. , 29.5])

In [16]:
# seqCPMG.sequences['awg'][1].print_info()

In [17]:
seqCPMG.write_seq()

MCAS 'my_cpmg' has been generated.


In [18]:
seqCPMG.run_sequence()

writing sequence my_cpmg on AWG_M8190A_qudi ch 1 took 3.8402 seconds
writing sequence my_cpmg on AWG_M8190A_qudi ch 2 took 3.6399 seconds


In [22]:
seqCPMG.stop_awgs()

In [267]:
seqCPMG._awg


<bound method MultiChSeq._awg of <hardware.awg.multi_channel_awg_seq.MultiChSeq object at 0x0000013CB86AD668>>

In [17]:
electron_pi = 1.4/2
tau = 1
pi_samples = round(electron_pi * 1e-6 / (1. / 12e9))
pi_half_samples = round(electron_pi / 2. * 1e-6 / (1. / 12e9))
print('tau ',tau )
print('pi_samples ',pi_samples)
wait = tau - pi_samples
wait_half = wait / 2.
print('wait_half1 ',wait_half)

if wait_half % 1 == 0:
    wait_half_2 = wait_half
else:
    wait_half_2 = int(wait_half) + 1
    wait_half = int(wait_half)

print('wait_half1_2 ',wait_half)

tau  1
pi_samples  8400
wait_half1  -4199.5
wait_half1_2  -4199


In [19]:

(electron_pi * 1e-6 / (1. / 12e9))


8400.0

In [20]:
1 / 0.0000833333333

12000.0000048

In [19]:
(2877 - 2329.92) / 2.8

195.38571428571427

In [20]:
1- (52/58)

0.10344827586206895

## TESTS

In [271]:

def awg_cpmg_test( tau):
    
    seq = mcas.MultiChSeq('my_cpmg', ch_dict={'awg': [1, 2]})
    phases = [0.,180.]
    
    awg_freq = 2697.000  # in MHz
    electron_amp = 0.2 # 
    electron_pi = 1.021/2 # us
    
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
#     double check
    
#     for tau in taus:
    for phase in phases:

        seq.start_new_segment('my_cpmg')
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                length_mus=electron_pi / 2.)
# 1X
        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)
# 2Y          
        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)
# 3X            
        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)
# 4Y            
        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)
# 5X            

        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)                
# 6Y 
        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)            
# 7X 
        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [90]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)                
# 8Y
        seq = awg_AddWait(seq, tau / 2.)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],phases = [0]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, tau / 2.)            
#RO    
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],
                            phases=[phase]),
                length_mus=electron_pi / 2.)
        seq.asc(length_mus=0.1)          
        seq = awg_AddReadout(seq)
    return seq





In [278]:
import time
seqTest = awg_cpmg_test(10)
seqTest.awg = awg
seqTest.write_seq()
print('-----')
begin = time.time()

seqTest.run_sequence()
end = time.time()
time.sleep (0.5)
seqTest.stop_awgs()

print(end-begin)

MCAS 'my_cpmg' has been generated.
-----
writing sequence my_cpmg on AWG_M8190A_qudi ch 1 took 0.1309 seconds
writing sequence my_cpmg on AWG_M8190A_qudi ch 2 took 0.1139 seconds
writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0090 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0100 seconds
0.2898592948913574


In [275]:
import time
time.time()

1585154984.9073546

In [295]:
print('-----')
begin = time.time()
fastcounter.pulsed.getData()
end = time.time()
print(end-begin)

-----
0.0009989738464355469
